<a href="https://colab.research.google.com/github/jjups96/mcd_ingenieria_de_caracteristicas/blob/main/Proyecto1/Proyecto1_VersionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este proyecto me interesa responder preguntas relacionadas con la pobreza y el crimen, particularmente como estan relacionados, por ello utilizaremos el polemico indice de marginacion, y datos recolectados por el C5. Para esta primera iteracion solo nos interesa el numero total. Dependiendo de los resultados del EDA, ya se tiene una fuente de informacion mas detallada.

Primero instalamos la dependencia que tiene pandas para leer documentos de excel, pero que por alguna razon no lo hace en automatico.

In [30]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importamos unas librerias basicas

In [31]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
import os
import zipfile
from os import path
import requests
import json

Primero descargamos el archivo xls de datos de marginacion para tenerlos a la mano

In [77]:
# pwd
print(os.getcwd())

#  Estos son los datos que vamos a descargar y donde vamos a guardarlos
marginacion_2020_url = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls"
marginacion_2020_archivo = "IMM_2020.xls"
incidencias911_2020_url = "http://apps.sspsonora.gob.mx/Incidencia911/Analisis/obtTablaIncidentesMunicipio"
incidencias911_2020_archivo = "incidencias911_2020.csv"
resultado_archivo = "resultado.csv"

subdir = "./data/"

/content


# Indice de marginacion

In [35]:
if not os.path.exists(marginacion_2020_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)

    resp = requests.get(marginacion_2020_url)
    output = open(subdir + marginacion_2020_archivo, 'wb')
    output.write(resp.content)
    output.close()

    with open(subdir + "info_IMM.txt", 'w') as f:
        f.write("Archivos sobre el indice de marginacion\n")
        info = """
        Archivo que contiene los indicadores socioeconómicos que conformaron cada 
        índice de marginación por municipio

        Pagina principal: https://www.gob.mx/conapo/documentos/indices-de-marginacion-2020-284372
        Nota tecnica del calculo y fuentes: https://www.gob.mx/cms/uploads/attachment/file/685354/Nota_te_cnica_IMEyM_2020.pdf
        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + marginacion_2020_url + "\n")
        f.write("Nombre: " + marginacion_2020_archivo + "\n")

Aunque ya lo tenemos descargado, para efectos didacticos descargamos solo la hoja que necesitamos con pandas usando la url

In [36]:
IMM_2020 = pd.read_excel('http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls', sheet_name='IMM_2020')

In [37]:
IMM_2020.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020
0,1,Aguascalientes,1001,Aguascalientes,948990,1.644738,20.367220,0.104799,0.113169,0.378610,0.591434,10.339530,7.523683,54.226594,60.318795,Muy bajo,0.944508
1,1,Aguascalientes,1002,Asientos,51536,3.526405,33.906364,2.650373,0.486448,0.858160,1.352430,22.942305,78.221049,78.565471,56.546071,Muy bajo,0.885433
2,1,Aguascalientes,1003,Calvillo,58250,4.491509,42.482450,0.365177,0.516760,0.800978,1.040411,19.219858,51.301288,79.259777,57.058251,Muy bajo,0.893453
3,1,Aguascalientes,1004,Cosío,17000,3.144867,27.696745,0.712855,0.577354,0.659833,1.030989,22.716866,65.470588,81.726369,57.114030,Muy bajo,0.894326
4,1,Aguascalientes,1005,Jesús María,129929,2.380588,26.692477,0.277034,0.354957,0.860426,1.312652,16.404575,37.164143,56.748753,59.011762,Muy bajo,0.924042


Ahora que ya tenemos los datos de todos los municipios, realizaremos un filtro para solo conserver los datos de Sonora. Con dicho filtro dos columnas se vuelven innecesarias

In [38]:
IMM_2020_Sonora = IMM_2020[IMM_2020['NOM_ENT'] == 'Sonora']
IMM_2020_Sonora = IMM_2020_Sonora.drop(['NOM_ENT','CVE_ENT'], axis=1)
IMM_2020_Sonora = IMM_2020_Sonora.reset_index(drop=True)

In [39]:
IMM_2020_Sonora.head()

,CVE_MUN,NOM_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020
0,26001,Aconchi,2563,2.691722,37.125444,0.780336,0.858369,0.702302,0.312134,16.329113,100.000000,66.869301,57.175873,Muy bajo,0.895295
1,26002,Agua Prieta,91929,1.488203,26.019518,0.203682,1.035895,0.618724,0.946196,17.610458,0.979016,81.065659,58.632120,Muy bajo,0.918097
2,26003,Alamos,24976,8.107527,41.656634,7.845339,4.087117,5.190117,10.769724,26.759377,56.113869,75.732267,53.436432,Medio,0.836740
3,26004,Altar,9492,3.433923,35.089742,1.144674,0.932697,1.028087,1.017920,23.018867,11.093552,89.689880,57.254916,Muy bajo,0.896532
4,26005,Arivechi,1177,3.265766,45.912654,0.256630,3.250642,0.085543,1.881950,10.997442,100.000000,60.227273,56.732817,Muy bajo,0.888357


In [41]:
IMM_2020_Sonora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CVE_MUN   72 non-null     int64  
 1   NOM_MUN   72 non-null     object 
 2   POB_TOT   72 non-null     int64  
 3   ANALF     72 non-null     float64
 4   SBASC     72 non-null     float64
 5   OVSDE     72 non-null     float64
 6   OVSEE     72 non-null     float64
 7   OVSAE     72 non-null     float64
 8   OVPT      72 non-null     float64
 9   VHAC      72 non-null     float64
 10  PL.5000   72 non-null     float64
 11  PO2SM     72 non-null     float64
 12  IM_2020   72 non-null     float64
 13  GM_2020   72 non-null     object 
 14  IMN_2020  72 non-null     float64
dtypes: float64(11), int64(2), object(2)
memory usage: 8.6+ KB


De lo anterior, podemos observar que en efecto tenemos los 72 municipios de Sonora

# Incidencias 911 Sonora

Para estos datos es necesario mas contexto, si buscamos informacion de seguridad o llamas al 911 podemos encontrar esta [pagina del gobierno ](https://www.gob.mx/911/documentos/estadistica-nacional-del-numero-de-atencion-de-llamadas-de-emergencia-9-1-1-111029)con unas presentaciones muy bonitas. La fuente nunca es cita directamente, solo se hace mencion de un **Catálogo Nacional de Incidentes de Emergencia**. Buscando mas al respecto llegue a esta la pagina de datos abiertos del [ Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública](https://https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva) donde podemos encontrar un link que lleva aun zip con las [Cifras de Incidencia Delictiva Municipal](https://drive.google.com/file/d/1R5zd3opvicQ4uR4-eDitLRoywNlFoqjM/view), aqui se pueden encontrar cifras mas especificas seperadas por año.

Sin embargo para la informacion que requerimos usar en esta ocasion que no es tan especifica, por ello usamos de fuente la web app de la [app web de la secretaria de seguridad publica del gobierno se Sonora](https://http://apps.sspsonora.gob.mx/Incidencia911/Analisis/Index), hay que usar los recursos locales, ademas que anteriormente ya se descargo un archivo que estaba en linea y como parte de la tarea es mejor mostrar diferentes maneras de descargar datos, por ejemplo mediante el llamado a una API.

In [46]:
url = 'http://apps.sspsonora.gob.mx/Incidencia911/Analisis/obtTablaIncidentesMunicipio'


'[{"DataName":"INCIDENTES_MUNICIPIO","Data":[{"Municipio":"Hermosillo","Total":"31259"},{"Municipio":"Cajeme","Total":"14825"},{"Municipio":"San Luis Rio Colorado","Total":"11140"},{"Municipio":"Nogales","Total":"9274"},{"Municipio":"Navojoa","Total":"3539"},{"Municipio":"Guaymas","Total":"3056"},{"Municipio":"Agua Prieta","Total":"2183"},{"Municipio":"Puerto Peñasco","Total":"2110"},{"Municipio":"Caborca","Total":"1796"},{"Municipio":"Etchojoa","Total":"1053"},{"Municipio":"Huatabampo","Total":"1041"},{"Municipio":"Empalme","Total":"905"},{"Municipio":"Cananea","Total":"448"},{"Municipio":"Magdalena","Total":"405"},{"Municipio":"Bacum","Total":"335"},{"Municipio":"Benito Juarez","Total":"264"},{"Municipio":"Imuris","Total":"157"},{"Municipio":"Santa Ana","Total":"144"},{"Municipio":"San Ignacio Rio Muerto","Total":"89"},{"Municipio":"Alamos","Total":"75"},{"Municipio":"San Miguel de Horcasitas","Total":"57"},{"Municipio":"Altar","Total":"43"},{"Municipio":"Benjamin Hill","Total":"38"}

In [76]:
if not os.path.exists(incidencias911_2020_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)

    headers = {'Content-type': 'application/json', 'dataType': 'json'}
    body = {'_anio_inicial':'2020','_mes_inicial':'1','_anio_final':'2020','_mes_final':'12','_id_municipio':'0','_id_clasificacion':'0','_id_incidente':'0'}
    response = requests.post(incidencias911_2020_url, json = body,headers= headers)
    response.text

    incidencias911_Sonora_json = json.loads(response.text)
    incidencias911_2020_Sonora = pd.DataFrame.from_dict(incidencias911_Sonora_json[0]['Data'])
    incidencias911_2020_Sonora.to_csv(subdir + incidencias911_2020_archivo)  

    with open(subdir + "info_911.txt", 'w') as f:
        f.write("Archivos sobre incidencias reporttadas al 911 en Sonora\n")
        info = """
        Archivo que contiene una tabla con el numero de incidencias reportadas al
        911 en el estado de Sonora, la aplicacion esta respaldada por el C5 del
        estado, pero fuera de eso no ha mucha mas informacion al respecto.
        En la libreta se agrego un comentario sobre cual podria ser la fuente 
        original de estos datos.

        Pagina principal: http://apps.sspsonora.gob.mx/Incidencia911/
        """ 
        f.write(info + '\n')
        f.write("Agregados nacionales descargados desde: " + incidencias911_2020_url + "\n")
        f.write("Nombre: " + incidencias911_2020_archivo + "\n")

In [67]:
incidencias911_2020_Sonora.head()

,Municipio,Total
0,Hermosillo,31259
1,Cajeme,14825
2,San Luis Rio Colorado,11140
3,Nogales,9274
4,Navojoa,3539


In [68]:
incidencias911_2020_Sonora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Municipio  55 non-null     object
 1   Total      55 non-null     object
dtypes: object(2)
memory usage: 1008.0+ bytes


In [69]:
incidencias911_2020_Sonora[["Total"]] = incidencias911_2020_Sonora[["Total"]].apply(pd.to_numeric)

In [70]:
incidencias911_2020_Sonora[incidencias911_2020_Sonora['Total']==0].shape

(0, 2)

De lo anterior podemos ver que no hay municipios con valor cero, esto puede significar que los valores con NaN, son en realidad cero, en cualquiere caso sustituir los valores NaN con cero dara como resultado una tabla mas tidy, y no introducira ruido para la posterior exploracion

Finalmente realizamos un merge

In [74]:
result = IMM_2020_Sonora.set_index('NOM_MUN').join(incidencias911_2020_Sonora.set_index('Municipio'))
result.rename(columns = {'Total':'INS_911'}, inplace = True)
result['INS_911'] = result['INS_911'].fillna(0)

In [75]:
result

,CVE_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020,INS_911
NOM_MUN,,,,,,,,,,,,,,,
Aconchi,26001,2563,2.691722,37.125444,0.780336,0.858369,0.702302,0.312134,16.329113,100.000000,66.869301,57.175873,Muy bajo,0.895295,1.0
Agua Prieta,26002,91929,1.488203,26.019518,0.203682,1.035895,0.618724,0.946196,17.610458,0.979016,81.065659,58.632120,Muy bajo,0.918097,2183.0
Alamos,26003,24976,8.107527,41.656634,7.845339,4.087117,5.190117,10.769724,26.759377,56.113869,75.732267,53.436432,Medio,0.836740,75.0
Altar,26004,9492,3.433923,35.089742,1.144674,0.932697,1.028087,1.017920,23.018867,11.093552,89.689880,57.254916,Muy bajo,0.896532,43.0
Arivechi,26005,1177,3.265766,45.912654,0.256630,3.250642,0.085543,1.881950,10.997442,100.000000,60.227273,56.732817,Muy bajo,0.888357,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,26068,1043,2.441860,41.811847,3.464870,1.732435,1.924928,1.924928,7.200000,100.000000,70.891089,56.594601,Muy bajo,0.886193,3.0
Yécora,26069,4793,9.025788,51.576835,1.761376,3.564689,6.689033,12.350598,19.512196,100.000000,79.286336,53.187621,Medio,0.832844,0.0
General Plutarco Elías Calles,26070,13627,3.092565,31.458325,0.536633,0.767683,1.043452,1.281956,21.671598,16.049020,88.378327,57.533394,Muy bajo,0.900893,0.0


Finalmente guardamos el dataframe final.

In [78]:
result.to_csv(subdir + resultado_archivo)  